# Compliance Campaign Analysis

This project uses advanced SQL querying within a Kaggle Notebook to analyze the Bank Marketing dataset. The primary goal is to evaluate customer segmentation, campaign efficiency, and compliance risks based on historical marketing outreach. This notebook is structured into the following key sections:

1. Data Exploration and Setup  
2. SQL Analysis  
    - Conversion Analysis by Job Role  
    - Conversion Analysis by Age Group  
    - Over-Targeted Job Segments  
    - Call Duration Insights by Job  
    - Campaign Effectiveness by Education Level  
    - Financial Risk Profiling (Housing, Loan, and Default)  
3. Conclusion  

In [27]:
import pandas as pd

# Load the dataset with the correct delimiter
df = pd.read_csv("/kaggle/input/bank-marketing/bank-additional-full.csv", sep=';')

# Preview the first few rows
df.head()


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [28]:
!pip install -q pandasql
from pandasql import sqldf

# SQL query function
pysqldf = lambda q: sqldf(q, globals())


In [29]:
# Conversion count by job role
query = """
SELECT 
    job,
    COUNT(*) AS total_contacts,
    SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
    ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
FROM df
GROUP BY job
ORDER BY conversion_rate DESC
"""

pysqldf(query)

,job,total_contacts,conversions,conversion_rate
0,student,875,275,31.43
1,retired,1720,434,25.23
2,unemployed,1014,144,14.20
3,admin.,10422,1352,12.97
4,management,2924,328,11.22
5,unknown,330,37,11.21
6,technician,6743,730,10.83
7,self-employed,1421,149,10.49
8,housemaid,1060,106,10.00
9,entrepreneur,1456,124,8.52


In [30]:
#Conversion Rate by Age Group
query = """
SELECT 
    CASE 
        WHEN age < 25 THEN 'Under 25'
        WHEN age BETWEEN 25 AND 34 THEN '25–34'
        WHEN age BETWEEN 35 AND 44 THEN '35–44'
        WHEN age BETWEEN 45 AND 54 THEN '45–54'
        WHEN age BETWEEN 55 AND 64 THEN '55–64'
        ELSE '65+'
    END AS age_group,
    COUNT(*) AS total_contacts,
    SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
    ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
FROM df
GROUP BY age_group
ORDER BY age_group
"""

pysqldf(query)

,age_group,total_contacts,conversions,conversion_rate
0,25–34,13686,1666,12.17
1,35–44,13500,1168,8.65
2,45–54,8704,753,8.65
3,55–64,3567,484,13.57
4,65+,663,313,47.21
5,Under 25,1068,256,23.97


The conversion rates shown in the age group analysis reflect the percentage of successful conversions **only among those contacted** in each group. These values are not normalized by the size of the overall age group in the dataset.

This means that a group with a small number of contacts, such as the `65+` age group can appear to have a disproportionately high conversion rate due to the small sample size.

To get a clearer comparison across age groups, a normalized conversion metric has been added, which accounts for each group's size in the full dataset.

In [31]:
query = """
-- Step 1: Creating age groups
WITH age_grouped AS (
    SELECT *,
        CASE 
            WHEN age < 25 THEN 'Under 25'
            WHEN age BETWEEN 25 AND 34 THEN '25–34'
            WHEN age BETWEEN 35 AND 44 THEN '35–44'
            WHEN age BETWEEN 45 AND 54 THEN '45–54'
            WHEN age BETWEEN 55 AND 64 THEN '55–64'
            ELSE '65+'
        END AS age_group
    FROM df
),

-- Step 2: Tagging high-pressure contacts
pressure_flagged AS (
    SELECT *,
        CASE 
            WHEN campaign > 3 THEN 'High Pressure'
            ELSE 'Normal'
        END AS pressure_level
    FROM age_grouped
),

-- Step 3: Group by age and pressure level
conversion_summary AS (
    SELECT 
        age_group,
        pressure_level,
        COUNT(*) AS total_contacts,
        SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
        ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
    FROM pressure_flagged
    GROUP BY age_group, pressure_level
)

-- Step 4: Final filtered result
SELECT *
FROM conversion_summary
WHERE conversion_rate > 10
ORDER BY age_group, pressure_level
"""

pysqldf(query)

,age_group,pressure_level,total_contacts,conversions,conversion_rate
0,25–34,Normal,11222,1463,13.04
1,55–64,Normal,2844,415,14.59
2,65+,High Pressure,73,23,31.51
3,65+,Normal,590,290,49.15
4,Under 25,High Pressure,182,22,12.09
5,Under 25,Normal,886,234,26.41


### Conversion Rate by Age Group and Contact Pressure

This table breaks down conversion rates by **age group** and **marketing pressure level**, where "High Pressure" means the client was contacted more than 3 times.

Observations:
- **Younger age groups (Under 25)** show high conversion rates under **Normal pressure** (26.41%) compared to High Pressure (12.09%), suggesting they respond better to lighter touchpoints.
- **Older clients (65+)** have extremely high conversion rates (49.15%) when contacted normally, but their total volume is very low — so this result should be interpreted with caution.
- **Mid age groups (25–34, 55–64)** also perform better under Normal pressure, reinforcing that repeated contact may not improve success.

**Conclusion:** Normal contact pressure generally results in better conversion rates across all age groups. High pressure may reduce effectiveness, especially in younger segments.

In [32]:
query = """
--- Use Case: Identify over-targeted job segments that may indicate aggressive or non-compliant outreach behavior
SELECT job, 
       COUNT(*) AS total_contacts,
       SUM(CASE WHEN campaign > 3 THEN 1 ELSE 0 END) AS high_pressure_contacts,
       ROUND(100.0 * SUM(CASE WHEN campaign > 3 THEN 1 ELSE 0 END) / COUNT(*), 2) AS high_pressure_pct
FROM df
GROUP BY job
ORDER BY high_pressure_pct DESC
"""
pysqldf(query)

,job,total_contacts,high_pressure_contacts,high_pressure_pct
0,unknown,330,68,20.61
1,housemaid,1060,213,20.09
2,self-employed,1421,282,19.85
3,admin.,10422,2018,19.36
4,technician,6743,1280,18.98
5,services,3969,735,18.52
6,unemployed,1014,187,18.44
7,blue-collar,9254,1682,18.18
8,management,2924,508,17.37
9,retired,1720,295,17.15


In [33]:
query = """
--- Use Case: To determine if follow-up calls lead to better conversions or if first-touch outreach is more effective.
SELECT 
    previous,
    COUNT(*) AS total,
    SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
    ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END)/COUNT(*), 2) AS conversion_rate
FROM df
GROUP BY previous
ORDER BY previous
"""
pysqldf(query)

,previous,total,conversions,conversion_rate
0,0,35563,3141,8.83
1,1,4561,967,21.20
2,2,754,350,46.42
3,3,216,128,59.26
4,4,70,38,54.29
5,5,18,13,72.22
6,6,5,3,60.00
7,7,1,0,0.00


In [34]:
query = """
-- Use Case: Identify which job titles convert with shorter or longer calls

SELECT 
    job,
    ROUND(AVG(duration), 2) AS avg_call_duration,
    COUNT(*) AS total_contacts,
    SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
    ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
FROM df
GROUP BY job
ORDER BY avg_call_duration DESC
"""
pysqldf(query)

,job,avg_call_duration,total_contacts,conversions,conversion_rate
0,student,283.68,875,275,31.43
1,retired,273.71,1720,434,25.23
2,blue-collar,264.54,9254,638,6.89
3,self-employed,264.14,1421,149,10.49
4,entrepreneur,263.27,1456,124,8.52
5,services,258.40,3969,323,8.14
6,management,257.06,2924,328,11.22
7,admin.,254.31,10422,1352,12.97
8,housemaid,250.45,1060,106,10.00
9,technician,250.23,6743,730,10.83


In [35]:
query = """
-- Use Case: Analyze potential biases in campaign effectiveness by education level to ensure ethical outreach practices and optimize audience targeting

SELECT education, 
       COUNT(*) AS total,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
       ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END)/COUNT(*), 2) AS conversion_rate
FROM df
GROUP BY education
"""
pysqldf(query)

,education,total,conversions,conversion_rate
0,basic.4y,4176,428,10.25
1,basic.6y,2292,188,8.20
2,basic.9y,6045,473,7.82
3,high.school,9515,1031,10.84
4,illiterate,18,4,22.22
5,professional.course,5243,595,11.35
6,university.degree,12168,1670,13.72
7,unknown,1731,251,14.50


In [36]:
query = """
-- Use Case: Assessing financial risk profile in outreach to minimize exposure
SELECT housing, loan, "default",
       COUNT(*) AS total,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
       ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
FROM df
GROUP BY housing, loan, "default"
"""
pysqldf(query)

,housing,loan,default,total,conversions,conversion_rate
0,no,no,no,12606,1569,12.45
1,no,no,unknown,3457,183,5.29
2,no,no,yes,2,0,0.00
3,no,yes,no,2008,250,12.45
4,no,yes,unknown,549,24,4.37
5,unknown,unknown,no,763,96,12.58
6,unknown,unknown,unknown,227,11,4.85
7,yes,no,no,14243,1909,13.40
8,yes,no,unknown,3641,189,5.19
9,yes,no,yes,1,0,0.00


In [37]:
def conversion_rate_by_job():
    query = """
    -- Use Case: Stored Procedure of conversion rate by job
    SELECT 
        job,
        COUNT(*) AS total_contacts,
        SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
        ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
    FROM df
    GROUP BY job
    ORDER BY conversion_rate DESC
    """
    return pysqldf(query)

In [38]:
conversion_rate_by_job()

,job,total_contacts,conversions,conversion_rate
0,student,875,275,31.43
1,retired,1720,434,25.23
2,unemployed,1014,144,14.20
3,admin.,10422,1352,12.97
4,management,2924,328,11.22
5,unknown,330,37,11.21
6,technician,6743,730,10.83
7,self-employed,1421,149,10.49
8,housemaid,1060,106,10.00
9,entrepreneur,1456,124,8.52


In [39]:
def conversion_by_month():
    query = """
    -- Use Case: Stored Procedure to understand seasonality trends in conversions
    SELECT 
        month,
        COUNT(*) AS total_contacts,
        SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
        ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
    FROM df
    GROUP BY month
    ORDER BY conversion_rate DESC
    """
    return pysqldf(query)

In [40]:
conversion_by_month()

,month,total_contacts,conversions,conversion_rate
0,mar,546,276,50.55
1,dec,182,89,48.90
2,sep,570,256,44.91
3,oct,718,315,43.87
4,apr,2632,539,20.48
5,aug,6178,655,10.60
6,jun,5318,559,10.51
7,nov,4101,416,10.14
8,jul,7174,649,9.05
9,may,13769,886,6.43


In [41]:
def top_duration_segments():
    query = """
    -- Use Case: stored procedure to identify groups with the longest conversations that lead to conversions
    SELECT job, education, AVG(duration) AS avg_duration
    FROM df
    WHERE y = 'yes'
    GROUP BY job, education
    ORDER BY avg_duration DESC
    LIMIT 10
    """
    return pysqldf(query)

In [42]:
top_duration_segments()

,job,education,avg_duration
0,unknown,professional.course,2029.000000
1,housemaid,basic.6y,1143.000000
2,self-employed,basic.9y,1012.277778
3,services,basic.4y,942.428571
4,housemaid,basic.9y,907.000000
5,unknown,basic.6y,878.000000
6,entrepreneur,illiterate,838.000000
7,unemployed,basic.6y,834.750000
8,admin.,basic.6y,815.500000
9,blue-collar,basic.6y,801.345794


In [43]:
def risk_profile_conversion():
    query = """
    -- Use Case: stored procedure to assess financial risk profile vs. conversion behavior
    SELECT 
        housing, 
        loan, 
        "default",
        COUNT(*) AS total, 
        SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
        ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END)/COUNT(*), 2) AS conversion_rate
    FROM df
    GROUP BY housing, loan, "default"
    ORDER BY conversion_rate DESC
    """
    return pysqldf(query)

In [44]:
risk_profile_conversion()

,housing,loan,default,total,conversions,conversion_rate
0,yes,no,no,14243,1909,13.40
1,unknown,unknown,no,763,96,12.58
2,yes,yes,no,2968,373,12.57
3,no,no,no,12606,1569,12.45
4,no,yes,no,2008,250,12.45
5,no,no,unknown,3457,183,5.29
6,yes,no,unknown,3641,189,5.19
7,yes,yes,unknown,723,36,4.98
8,unknown,unknown,unknown,227,11,4.85
9,no,yes,unknown,549,24,4.37


In [45]:
def conversion_by_marital():
    query = """
    -- Use Case: Audit if marital status impacts campaign conversions
    SELECT 
        marital,
        COUNT(*) AS total_contacts,
        SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS conversions,
        ROUND(100.0 * SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS conversion_rate
    FROM df
    GROUP BY marital
    ORDER BY conversion_rate DESC
    """
    return pysqldf(query)

In [46]:
conversion_by_marital()

,marital,total_contacts,conversions,conversion_rate
0,unknown,80,12,15.00
1,single,11568,1620,14.00
2,divorced,4612,476,10.32
3,married,24928,2532,10.16
